## Preliminaries


This section import and load the necessary dependencies.


In [ ]:
# import Python libaries 
import os
import torch
from tqdm.notebook import tqdm
import pandas as pd
import glob

In [ ]:
# import local libraries
from scripts import fgsm
from scripts.utils import open_img_as_tensor
from scripts.face_detectors import MediaPipe, YuNet, YoloFace
from scripts.facesegmentor import FaceSegementor
from scripts import image_attributes

In [ ]:
# settings
image_attributes.save_color_images = False
image_attributes.save_lbp_images = False
image_attributes.save_gradient_images = False

## Face Segmentor


This section loads and instantiates the face segmentor from the `facesegmentor.py` python file found under the scripts folder.


In [ ]:
# instantiate face segmentor
faceseg = FaceSegementor()

## Face Detector Model


This section loads and instantiates the face detector from the `face_detectors.py` python file found under the scripts folder.


In [ ]:
# instantiate face detectors
yf = YoloFace()
yn = YuNet
mp = MediaPipe

## Configuration


The labels for the different perturbation regions (i.e. bbox and face region) are added to a list. Moreover, the models (i.e. YoloFace, YuNet, and MediaPipe) are to be stored under a dictionary. In addition, a label is also generated for each combination of perturbation region with the face detection models through a function.


In [ ]:
label_regions = ["bbox", "face"] # perturbation regions

# dictionaries of face detectors
models = {
    "yf": YoloFace(),
    "yn": YuNet(),
    "mp": MediaPipe()
}

# a function that retrieves the label name based on the face detection and perturbation region
def get_label(label_model, label_region):
    return "e_" + label_region + "_" + label_model

## Input Folder


This section defines the path to the folders containing the images that will be included in the dataset


In [ ]:
folders = [r"./input"]

## Dataset Creation


The pipeline for the creation of the dataset is outlined below


In [ ]:
df_dataset = pd.DataFrame()

#loops through the list of folders
for folder in tqdm(folders, desc="All folders progress"):

    #loops through the images within the folder
    for file_path in tqdm(glob.glob(os.path.join(folder, "*.jpg")), desc="Current folder progress"):

        input_img = open_img_as_tensor(file_path) #open the image as tensor

        #retrieve the image features, gradient noise, bounding box information, and mask for the face region
        feats, grads, bboxes, masks = image_attributes.get_features(file_path, face_segmentor=faceseg)

        # iterate through the different perturbation regions
        for label_region in label_regions:

            # if the perturbation region is for bounding box, create a mask for the bounding box region
            if label_region == "bbox":
                for i in range(len(masks)):
                    bbox_mask = masks[i]
                    bbox_mask[..., bboxes[i][1]:bboxes[i][3], bboxes[i][0]:bboxes[i][2]] = 1
                    masks[i] = bbox_mask
            
            # determine the minimum perturbation parameter for different face detectors
            for model in models:
                e_mins = []
                # search for the minimum perturbation parameter for the detected faces
                for data_grad, mask, bbox in zip(grads, masks, bboxes):
                    e_mins.append(fgsm.binary_search(input_img, data_grad, models[model], mask, bbox))
                label_name = get_label(model, label_region) # 
                feats[label_name] = e_mins #save the label in the dictionary
    
    # transform the list of dictionaries into a dataframe
    df_dataset = pd.concat([df_dataset, feats], ignore_index=True)

Once the generation for the dataset has been finished, save the resulting `DataFrame` as a `.csv` file named `"min_epsilon_dataset.csv"`


In [ ]:
df_dataset.to_csv("min_epsilon_dataset.csv")